## Deploying Voting ensemble Sklearn models on SageMaker

In [1]:
!rm modelscript_ensemble_sklearn.py
!rm *.joblib
!src

/bin/sh: src: command not found


In [2]:
# Voting Ensemble for Classification
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# Get Data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# Fit
kfold = model_selection.KFold(n_splits=10)
# create the sub models
estimators = []
model1 = LogisticRegression(solver='newton-cg')
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)

ensemble.fit(X,Y)
results = model_selection.cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())

0.7669343814080657


### Save model files

In [3]:
!pip install joblib -q
from joblib import dump

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
for estimator in ensemble.estimators:
    print(estimator[0])
    dump(estimator[1],estimator[0]+'.joblib')

dump(ensemble,'ensemble.joblib')

logistic
cart
svm


['ensemble.joblib']

In [5]:
!ls *.joblib

cart.joblib  ensemble.joblib  logistic.joblib  svm.joblib


In [6]:
ensemble.predict(X[0].reshape(1,8))

array([1.])

## Step 1 : Write a model transform script

#### Make sure you have a ...

- "load_model" function
    - input args are model path
    - returns loaded model object
    - model name is the same as what you saved the model file as (see above step)
<br><br>
- "predict" function
    - input args are the loaded model object and a payload
    - returns the result of model.predict
    - make sure you format it as a single (or multiple) string return inside a list for real time (for mini batch)
    - from a client, a list  or string or np.array that is sent for prediction is interpreted as bytes. Do what you have to for converting back to list or string or np.array
    - return the error for debugging


In [7]:
%%writefile modelscript_ensemble_sklearn.py
import sklearn
from joblib import load
import numpy as np
import os

#Return loaded model
def load_model(modelpath):
    print(modelpath)
    
    # Either load individually
    print("loading individuals")
    logistic = load(os.path.join(modelpath,'logistic.joblib'))
    cart = load(os.path.join(modelpath,'cart.joblib'))
    svm = load(os.path.join(modelpath,'svm.joblib'))
    
    # Or load the entire ensemble
    print("loading ensemble")
    ensemble = load(os.path.join(modelpath,'ensemble.joblib'))
    print("loaded")
    return ensemble

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        # locally, payload may come in as an np.ndarray
        if type(payload)==np.ndarray:
            out = [str(model.predict(payload.reshape((1,8))))]
        # in remote / container based deployment, payload comes in as a stream of bytes
        else:

            out = [str(model.predict(np.frombuffer(payload).reshape((1,8))))]
    except Exception as e:
        out = [type(payload),str(e)] #useful for debugging!
    
    return out

Writing modelscript_ensemble_sklearn.py


## Does this work locally? (not "_in a container locally_", but _actually_ in local)

In [8]:
from modelscript_ensemble_sklearn import *
model = load_model('.')

.
loading individuals
loading ensemble
loaded


In [9]:
predict(model,X[0])

['[1.]']

### ok great! Now let's install ezsmdeploy
In some cases, installs fail due to an existing package installed called greenlet.
This is not a direct dependency of ezsmdeploy but interferes with the installation. 
To fix this, either install in a virtualenv as seen above, or do:
pip install ezsmdeploy[locust] --ignore-installed greenlet

In [10]:
!pip uninstall -y ezsmdeploy
!pip install ezsmdeploy==1.0.3

Found existing installation: ezsmdeploy 1.0.3
Uninstalling ezsmdeploy-1.0.3:
  Successfully uninstalled ezsmdeploy-1.0.3
  Using cached ezsmdeploy-1.0.3-py3-none-any.whl (23 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [11]:
import ezsmdeploy

#### If you have been running other inference containers in local mode, stop existing containers to avoid conflict

In [12]:
!docker container stop $(docker container ls -aq) >/dev/null

## Deploy locally

#### tar all models

In [13]:
!rm model.tar.gz
!find ./ -name "*.joblib" | tar -czf model.tar.gz -T -

In [14]:
ez = ezsmdeploy.Deploy(model = 'model.tar.gz', # if you intend to add models later, pass model as list, otherwise str
                  script = 'modelscript_ensemble_sklearn.py',
                  requirements = ['scikit-learn=='+sklearn.__version__,'numpy','joblib'], #or pass in the path to requirements.txt
                  instance_type = 'local',
                  wait = True)

0:00:00.122748 | compressed model(s)
0:00:00.192377 | uploaded model tarball(s) ; check returned modelpath
0:00:00.193133 | added requirements file
0:00:00.194746 | added source file
0:00:00.195986 | added Dockerfile
0:00:00.197790 | added model_handler and docker utils
0:00:00.197868 | building docker container
0:00:40.282419 | built docker container
2m●∙∙ 

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


0:00:40.438909 | created model(s). Now deploying on local
∙●∙ Attaching to tmpu4upynvz_algo-1-ru1bx_1
algo-1-ru1bx_1  | Starting the inference server with 32 workers.
∙∙● algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [9] [INFO] Starting gunicorn 20.0.4
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [9] [INFO] Using worker: gevent
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [13] [INFO] Booting worker with pid: 13
∙∙∙ algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [14] [INFO] Booting worker with pid: 14
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [15] [INFO] Booting worker with pid: 15
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [16] [INFO] Booting worker with pid: 16
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [17] [INFO] Booting worker with pid: 17
∙∙∙ algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] [19] [INFO] Booting worker with pid: 19
algo-1-ru1bx_1  | [2021-02-10 19:35:46 +0000] 

## Test containerized version locally

In [15]:
out = ez.predictor.predict(X[0].reshape((1,8)).tobytes()).decode()
out

algo-1-ru1bx_1  | received input data
algo-1-ru1bx_1  | b'\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x80b@\x00\x00\x00\x00\x00\x00R@\x00\x00\x00\x00\x00\x80A@\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\xcc\xcc\xcc\xcc@@\xaa\xf1\xd2Mb\x10\xe4?\x00\x00\x00\x00\x00\x00I@'
algo-1-ru1bx_1  | /opt/ml/model
algo-1-ru1bx_1  | loading individuals
algo-1-ru1bx_1  | loading ensemble
algo-1-ru1bx_1  | loaded
algo-1-ru1bx_1  | predictions from model
algo-1-ru1bx_1  | ['[1.]']


'[1.]'

algo-1-ru1bx_1  | 172.18.0.1 - - [10/Feb/2021:19:35:53 +0000] "POST /invocations HTTP/1.1" 200 4 "-" "python-urllib3/1.26.2"


In [16]:
!docker container stop $(docker container ls -aq) >/dev/null

algo-1-ru1bx_1  | [2021-02-10 19:36:06 +0000] [9] [INFO] Handling signal: term
tmpu4upynvz_algo-1-ru1bx_1 exited with code 0
Aborting on container exit...


## Deploy on SageMaker

In [17]:
ezonsm = ezsmdeploy.Deploy(model = 'model.tar.gz', 
                  script = 'modelscript_ensemble_sklearn.py',
                  requirements = ['scikit-learn=='+sklearn.__version__,'numpy','joblib'], #or pass in the path to requirements.txt
                  wait = True)

0:00:00.122377 | compressed model(s)
0:00:00.376112 | uploaded model tarball(s) ; check returned modelpath
0:00:00.377001 | added requirements file
0:00:00.378543 | added source file
0:00:00.380053 | added Dockerfile
0:00:00.381657 | added model_handler and docker utils
0:00:00.381762 | building docker container
0:00:40.467313 | built docker container
2m∙●∙ 

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


0:00:40.655411 | created model(s). Now deploying on ml.m5.xlarge
0:07:12.764312 | deployed model
0:07:12.765088 | estimated cost is $0.3 per hour
0:07:12.765247 | Done! ✔ 


In [ ]:
# !./src/build-docker.sh test

In [20]:
out = ezonsm.predictor.predict(X[0].reshape((1,8)).tobytes()).decode()
out

'[1.]'

In [21]:
ezonsm.predictor.delete_endpoint()